# 直接基于bar

In [1]:
from partition import *
import pretty_midi
import muspy
import pandas as pd
from tqdm import tqdm

In [2]:
index = pd.read_csv('./partition/index.csv')
len(index)

909

In [3]:
index

,song_id,name,artist,modify_times,num_beats_per_measure,num_quavers_per_beat,time_signature
0,1,123我爱你,新乐尘符,3,2,2,4/4
1,2,123木头人,黑涩会美眉,4,2,2,4/4
2,3,9420,麦小兜,2,2,2,4/4
3,4,Dear Friend,顺子,2,2,2,4/4
4,5,I Believe,范逸臣,1,2,2,4/4
...,...,...,...,...,...,...,...
904,905,小城故事,邓丽君,5,2,2,4/4
905,906,山不转水转,那英,3,2,2,4/4
906,907,小龙人,小龙人,3,2,2,4/4
907,908,小乌龟,张栋梁,1,2,2,4/4


## Section

In [4]:
for song_id in tqdm(index[index['time_signature'] == '4/4']['song_id']):
    partition_by_sections(song_id)

100%|██████████| 857/857 [03:09<00:00,  4.52it/s]


## Phrase

In [5]:
for song_id in tqdm(index[index['time_signature'] == '4/4']['song_id']):
    partition_by_phrase(song_id)

100%|██████████| 857/857 [04:25<00:00,  3.23it/s]


# 基于Downbeat/Chord等

In [1]:
from partition import *
import muspy
import pandas as pd
from tqdm import tqdm
import pretty_midi
import numpy as np
import random

index = pd.read_csv('./partition/index.csv')
len(index)

index

song_ids = index[index['time_signature'] == '4/4']['song_id'].tolist()
len(song_ids)

midi, pmidi, bar_window, chords = read_midi(1)

In [2]:
chords

[[40, 520, 'N'],
 [520, 1000, 'N'],
 [1000, 1480, 'N'],
 [1480, 1960, 'N'],
 [1960, 2920, 'B:maj'],
 [2920, 3880, 'C#:maj'],
 [3880, 4840, 'Bb:min'],
 [4840, 5800, 'Eb:min'],
 [5800, 6760, 'B:maj'],
 [6760, 7720, 'C#:maj'],
 [7720, 9640, 'F#:maj'],
 [9640, 10120, 'B:maj'],
 [10120, 10600, 'F#:maj7/5'],
 [10600, 11560, 'C#:maj'],
 [11560, 12040, 'Bb:min'],
 [12040, 12520, 'Bb:sus4'],
 [12520, 13480, 'Eb:min'],
 [13480, 13960, 'B:maj'],
 [13960, 14440, 'F#:maj7/5'],
 [14440, 15400, 'C#:maj'],
 [15400, 17320, 'F#:maj'],
 [17320, 18280, 'B:maj'],
 [18280, 19240, 'C#:maj'],
 [19240, 20200, 'Bb:min'],
 [20200, 21160, 'Eb:min'],
 [21160, 22120, 'B:maj'],
 [22120, 23080, 'C#:maj'],
 [23080, 25000, 'F#:maj'],
 [25000, 25960, 'B:maj'],
 [25960, 26920, 'C#:maj'],
 [26920, 27880, 'Bb:min'],
 [27880, 28840, 'Eb:min'],
 [28840, 29800, 'B:maj'],
 [29800, 30760, 'C#:maj'],
 [30760, 32680, 'F#:maj'],
 [32680, 33160, 'B:maj'],
 [33160, 33640, 'F#:maj7/5'],
 [33640, 34600, 'C#:maj'],
 [34600, 35080, 'Bb:

In [5]:
for t in midi:
    print(t.notes[0].time)

print()

for t in midi:
    print(t.notes[-1].time)

9160
1720
1960

130360
137560
138398


In [6]:
midi[0]

Track(program=0, is_drum=False, name='MELODY', notes=[Note(time=9160, pitch=61, duration=69, velocity=115), Note(time=9280, pitch=63, duration=45, velocity=109), Note(time=9400, pitch=66, duration=49, velocity=118), ...])

In [25]:
pmidi.instruments[0].notes

[Note(start=12.722190, end=12.818024, pitch=61, pitch_name=C#4, velocity=115),
 Note(start=12.888857, end=12.951357, pitch=63, pitch_name=D#4, velocity=109),
 Note(start=13.055523, end=13.123578, pitch=66, pitch_name=F#4, velocity=118),
 Note(start=13.222189, end=13.305522, pitch=68, pitch_name=G#4, velocity=112),
 Note(start=13.388855, end=13.498577, pitch=70, pitch_name=A#4, velocity=112),
 Note(start=13.722188, end=13.927743, pitch=66, pitch_name=F#4, velocity=112),
 Note(start=14.055520, end=14.295798, pitch=63, pitch_name=D#4, velocity=118),
 Note(start=14.388853, end=15.376350, pitch=68, pitch_name=G#4, velocity=115),
 Note(start=16.055515, end=16.283293, pitch=68, pitch_name=G#4, velocity=112),
 Note(start=16.388848, end=16.580514, pitch=65, pitch_name=F4, velocity=115),
 Note(start=16.722180, end=16.908291, pitch=61, pitch_name=C#4, velocity=115),
 Note(start=17.055513, end=17.824955, pitch=66, pitch_name=F#4, velocity=118),
 Note(start=18.055510, end=18.154121, pitch=61, pitch

In [34]:
def gen_analysis_df(song_id):
    midi, pmidi, bar_window, chords = read_midi(song_id)

    events = []

    # Note
    for t, track in enumerate(midi):
        for n, note in enumerate(track.notes):
            events.append({'onset': note.time, 'offset': note.time + note.duration,
                           'type': 'Note', 'track': track.name, 'name': 'pitch_{}'.format(pretty_midi.note_number_to_name(note.pitch))})

    # Chord
    for chord in chords:
        events.append({'onset': chord[0], 'offset': chord[1],
                       'type': 'Chord', 'track': 'N', 'name': chord[2]})

    # Bar
    num = chords[-1][1] // bar_window + 1
    for i in range(num + 1):
        events.append({'onset': bar_window * i, 'offset': bar_window*(i+1),
                       'type': 'Bar', 'track': 'N', 'name': 'Bar_{}'.format(i+1)})

    # Structure
    structure, _ = get_structure_labels(song_id)
    phrases = [p for sec in structure for p in sec]

    for p in phrases:
        events.append({'onset': bar_window * (p['from'] - 1), 'offset': bar_window * p['to'],
                       'type': 'Phrase', 'track': 'N', 'name': p['phrase']})

    events = sorted(events, key=lambda x: x['onset'])
    df = pd.DataFrame(events)

#     bar2phrase = dict()
#     for p in phrases:
#         for b in range(p['from'], p['to']+1):
#             bar2phrase['Bar_{}'.format(b)] = p['phrase']
#     for b in range(phrases[-1]['to'] + 1, num + 1):
#         bar2phrase['Bar_{}'.format(b)] = 'N'

#     phrases = []

    return df

In [35]:
df = gen_analysis_df(1)

In [38]:
# index = -50
index += 50
df[index:index+50]

,onset,offset,type,track,name
50,6760,6864,Note,BRIDGE,pitch_F#5
51,6760,7424,Note,PIANO,pitch_C#3
52,6760,6926,Note,PIANO,pitch_G#3
53,6760,7157,Note,PIANO,pitch_C#4
54,6760,7720,Chord,N,C#:maj
55,6880,6941,Note,BRIDGE,pitch_G#5
56,7000,7328,Note,BRIDGE,pitch_A#5
57,7000,7149,Note,PIANO,pitch_G#3
58,7120,7416,Note,PIANO,pitch_F4
59,7680,9600,Bar,N,Bar_5
